In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import requests

In [3]:
def get_sql_context_instance(spark_context):
	if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def aggregate_tags_count(new_values, total_sum):
    '''Ajoute le nombre de hashtags du delta à la valeur totale déjà existante'''
    return sum(new_values) + (total_sum or 0)

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        sql_context = get_sql_context_instance(rdd.context)
        # On met les données dans une ligne et on en fait une df
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        hashtags_df = sql_context.createDataFrame(row_rdd)
        hashtags_df.registerTempTable("hashtags")
        # Top 10 des hashtag
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
    except:
    	e = sys.exc_info()[0]
    	print("Error: %s" % e)


In [7]:


# Configuration Spark
conf = SparkConf()
conf.setAppName("Twitter-Analytics")
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

# Configuration Spark Streaming, update toutes les 2s
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint_TwitterApp")
dataStream = ssc.socketTextStream("localhost",9009) 

hashtags = dataStream.map(lambda x: (x, 1))
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

# On crée la DataFrames contenant les top 10 hashtags et on l'envoie à notre appli
tags_totals.foreachRDD(process_rdd)


ssc.start()
ssc.awaitTermination()

AttributeError: 'list' object has no attribute 'map'